# RDS server user data cleaning
This notebook was created to take advantage of the Jupyter visualizations when cleaning the data.<br>
The commands defined in this notebook will be then used in the **clean_user_data** method of the **DataCleaning** class.

 Performs the cleaning of the user data for 

1. NULL values
2. errors with dates
3. incorrectly typed values
4. rows filled with the wrong information

In [ ]:
# need to mount Google Drive to be able to load data from it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import pandas and the RDS table that needs cleaning
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/001. Data Science/005. Ai Core/014. Multinational Data Centralization Project/RDS_table.csv")

In [ ]:
df.head()
#df.columns

,Unnamed: 0,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     15320 non-null  int64 
 1   index          15320 non-null  int64 
 2   first_name     15299 non-null  object
 3   last_name      15299 non-null  object
 4   date_of_birth  15299 non-null  object
 5   company        15299 non-null  object
 6   email_address  15299 non-null  object
 7   address        15299 non-null  object
 8   country        15299 non-null  object
 9   country_code   15299 non-null  object
 10  phone_number   15299 non-null  object
 11  join_date      15299 non-null  object
 12  user_uuid      15299 non-null  object
dtypes: int64(2), object(11)
memory usage: 1.5+ MB


In [ ]:
df.duplicated().sum()

0

In [ ]:
df.isna().sum()

Unnamed: 0        0
index             0
first_name       21
last_name        21
date_of_birth    21
company          21
email_address    21
address          21
country          21
country_code     21
phone_number     21
join_date        21
user_uuid        21
dtype: int64

In [ ]:
#pd.set_option('display.max_rows', 500)
#for name in df['first_name'].unique():
#    print(name)

In [ ]:
# plot the "Unnamed" column
df["Unnamed: 0"].sample(100)

13176    13176
8344      8344
5550      5550
7930      7930
5820      5820
         ...  
9609      9609
6573      6573
740        740
15074    15074
494        494
Name: Unnamed: 0, Length: 100, dtype: int64

In [ ]:
# plot the "level_0" column
df["index"].sample(100)

7352      7360
4260      4263
1988      1989
1100      1101
7647      7656
         ...  
14872    14899
13106    13130
13802    13826
2919      2920
11070    11084
Name: index, Length: 100, dtype: int64

## To does
1. drop the "Unnamed: 0" and "index" columns
2. drop the rows containing NaN
3. Convert the DOB in datetime and leverage on it to determine if any of the dates is wrong

In [ ]:
df = df.drop(['Unnamed: 0', 'index'], axis = 1)

In [ ]:
df = df[df.notna().any(axis=1)]

In [ ]:
# not sure it is really worth modifying that
import datetime
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], infer_datetime_format=True, errors = 'coerce')

Remove the incorrect dates

In [ ]:
df = df[df['date_of_birth'].notna()]

In [ ]:
df['date_of_birth'].isna().sum()

0

DOB now contains zero errors

In [ ]:
pd.to_datetime(df['join_date'], infer_datetime_format=True, errors = 'coerce').isna().sum()

15

In [ ]:
df = df[df['join_date'].notna()]

In [ ]:
df['date_of_birth'].isna().sum()

0

In [ ]:
df['join_date'].isna().sum()

0

In [ ]:
df['country'].unique()

In [ ]:
df['country_code'].unique()

In [ ]:
# check if country and country_code coincide
print(len(df[(df['country'] == "Germany") & (df['country_code'] == "DE")]))
print(len(df[df['country'] == "Germany"]))
print(len(df[df['country_code'] == "DE"]))

In [ ]:
# check if country and country_code coincide
print(len(df[(df['country'] == "United Kingdom") & (df['country_code'] == "GB")]))
print(len(df[df['country'] == "United Kingdom"]))
print(len(df[df['country_code'] == "GB"]))

In [ ]:
# assign the correct country code
df.loc[(df['country'] == "United Kingdom") & (df['country_code'] != "GB"), 'country_code'] = 'GB'
# check if now the numbers are aligned
print(len(df[(df['country'] == "United Kingdom") & (df['country_code'] == "GB")]))
print(len(df[df['country'] == "United Kingdom"]))
print(len(df[df['country_code'] == "GB"]))

In [ ]:
print(len(df[(df['country'] == "United States") & (df['country_code'] == "US")]))
print(len(df[df['country'] == "United States"]))
print(len(df[df['country_code'] == "US"]))

In [ ]:
df['country'].value_counts()

In [ ]:
df[df['country'].isin(['I7G4DMDZOZ',
       'AJ1ENKS3QL', 'XGI7FM0VBJ', 'S0E37H52ON', 'XN9NGL5C0B',
       '50KUU3PQUF', 'EWE3U0DZIV', 'GMRBOMI0O1', 'YOTSVPRBQ7',
       '5EFAFD0JLI', 'PNRMPSYR1J', 'RQRB7RMTAD', '3518UD5CE8',
       '7ZNO5EBALT', 'T4WBZSW0XI'])]

In [ ]:
df[~df['country'].isin(['United Kingdom', 'Germany', 'United States'])]

These values are all wrong, let's drop them

In [ ]:
df = df[df['country'].isin(['United Kingdom', 'Germany', 'United States'])]

In [ ]:
df['country'].value_counts()

In [ ]:
df['country_code'].value_counts()

OK, I stop here! Tonight I better save the file!

In [ ]:
! pip install tabula-py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 51.7 MB/s eta 0:00:00


In [ ]:
import tabula
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
dfs = tabula.read_pdf(pdf_link, pages='all')

In [ ]:
# tabula extracts a list
type(dfs)

list

In [ ]:
len(dfs[0])

55

In [ ]:
import pandas as pd
df_2 = pd.concat(dfs)

In [ ]:
# this is what I have to input into the function:
import tabula
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
dfs = tabula.read_pdf(pdf_link, pages='all')
df2 = pd.concat(dfs)

https://stackoverflow.com/questions/65626278/using-tabula-py-to-read-table-without-header-from-pdf-format

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15309 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 598.0+ KB


In [ ]:
pd.set_option('display.max_rows', 500)
df2.head(10)

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
5,3506661913512980,11/23,JCB 16 digit,2003-08-25
6,377549437870679,07/27,American Express,2006-12-11
7,2321541881278150,02/29,Mastercard,1995-08-24
8,6011037917693140,02/24,Discover,1998-10-23
9,502049986008,07/23,Maestro,2011-04-30


In [ ]:
for index in df2.columns:
    num_nulls = (df2[index]=='NULL').sum() 
    print(f'The number of NULL in {index} is {num_nulls}' )

The number of NULL in card_number is 11
The number of NULL in expiry_date is 11
The number of NULL in card_provider is 11
The number of NULL in date_payment_confirmed is 11


it seems all the tables have the same number of nulls, 11 in total. Let's check they all belong to the same rows.

In [ ]:
df2 = df2[df2['card_number']!='NULL']

In [ ]:
df2.duplicated().sum()

0

there are no duplicated rows

In [ ]:
df2['card_provider'].unique()

array(['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit',
       'JCB 15 digit', 'Maestro', 'Mastercard', 'Discover',
       'VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit', 'NB71VBAHJE',
       'WJVMUO4QX6', 'JRPRLPIBZ2', 'TS8A81WFXV', 'JCQMU8FN85',
       '5CJH7ABGDR', 'DE488ORDXY', 'OGJTXI6X1H', '1M38DYQTZV',
       'DLWF2HANZF', 'XGZBYBYGUW', 'UA07L7EILH', 'BU9U947ZGV',
       '5MFWFBZRM9'], dtype=object)

In [ ]:
df2[df2['card_provider'].isin(['NB71VBAHJE',
       'WJVMUO4QX6', 'JRPRLPIBZ2', 'TS8A81WFXV', 'JCQMU8FN85',
       '5CJH7ABGDR', 'DE488ORDXY', 'OGJTXI6X1H', '1M38DYQTZV',
       'DLWF2HANZF', 'XGZBYBYGUW', 'UA07L7EILH', 'BU9U947ZGV',
       '5MFWFBZRM9'])]

,card_number,expiry_date,card_provider,date_payment_confirmed
2,VAB9DSB8ZM,NWS3P2W38H,NB71VBAHJE,GTC9KBWJO9
13,MOZOT5Q95V,8YJ3TYH6Z5,WJVMUO4QX6,DJIXF1AFAZ
9,K0084A9R99,ACT9K6ECRJ,JRPRLPIBZ2,H2PCQP4W50
28,Y8ITI33X30,WDWMN9TU45,TS8A81WFXV,XTD27ANR5Q
21,RNSCD8OCIM,VNLNMWPJII,JCQMU8FN85,7VGB4DA1WI
53,MIK9G2EMM0,4FI5GTUVYG,5CJH7ABGDR,RLQYRRYHPU
17,I4PWLWSIRJ,RF1ACW165R,DE488ORDXY,T008RE1ZR6
13,OMZSBN2XG3,6JJKS7R0WA,OGJTXI6X1H,7FL8EU9GBF
8,NB8JJ05D7R,XRPE6C4GS9,1M38DYQTZV,GD9PHJXQR4
7,G0EF4TS8C8,5VN8HOLMVE,DLWF2HANZF,WCK463ZO1Z


All the data with those weird card numbers are corrupted or unuseable. Let's remove them.

In [ ]:
df2 = df2[df2['card_provider'].isin(['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit',
       'JCB 15 digit', 'Maestro', 'Mastercard', 'Discover',
       'VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit'])]

In [ ]:
df2

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13
...,...,...,...,...
14,180036921556789,12/28,JCB 15 digit,1997-06-06
15,180018030448512,11/24,JCB 15 digit,2004-06-16
16,3569953313547220,04/24,JCB 16 digit,2020-02-05
17,4444521712606810,06/27,VISA 16 digit,2008-06-16


In [ ]:
# df2['date_payment_confirmed'] = pd.to_datetime(df2['date_payment_confirmed'], infer_datetime_format=True, errors = 'coerce')
pd.to_datetime(df2['date_payment_confirmed'], infer_datetime_format=True, errors = 'coerce').isna().sum()

0

No errors in the the date_payment_confirmed column

In [ ]:
df2['date_payment_confirmed'] = pd.to_datetime(df2['date_payment_confirmed'], infer_datetime_format=True, errors = 'coerce')

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15284 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   card_number             15284 non-null  object        
 1   expiry_date             15284 non-null  object        
 2   card_provider           15284 non-null  object        
 3   date_payment_confirmed  15284 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 1.1+ MB


## So to summarize

In [ ]:
df = df[df['card_number']!='NULL']
df = df[df['card_provider'].isin(['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit',
       'JCB 15 digit', 'Maestro', 'Mastercard', 'Discover',
       'VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit'])]